In [1]:
# Imports
import os
import re
import sys
import math
import h5py
import glob
import pickle
import random
import logging
import hashlib
import operator
from textwrap import wrap
from functools import reduce
from datetime import datetime
from multiprocessing import Pool
from itertools import islice, product
from collections import Counter, OrderedDict, defaultdict
from typing import Iterable, DefaultDict, Dict, List, Tuple, Optional, Callable

from sklearn import manifold
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import brier_score_loss, precision_score, recall_score, f1_score
from sklearn.calibration import calibration_curve
import seaborn as sns
from biosppy.signals import ecg
from scipy.ndimage.filters import gaussian_filter
from scipy import stats

from ml4cvd.TensorMap import TensorMap
from ml4cvd.metrics import concordance_index, coefficient_of_determination
from ml4cvd.defines import IMAGE_EXT, JOIN_CHAR, PDF_EXT, TENSOR_EXT, ECG_REST_LEADS, PARTNERS_DATETIME_FORMAT, PARTNERS_DATE_FORMAT

import matplotlib
matplotlib.use('Agg')  # Need this to write images from the GSA servers.  Order matters:
import matplotlib.pyplot as plt  # First import matplotlib, then use Agg, then import plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import NullFormatter
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

RECALL_LABEL = 'Recall | Sensitivity | True Positive Rate | TP/(TP+FN)'
FALLOUT_LABEL = 'Fallout | 1 - Specificity | False Positive Rate | FP/(FP+TN)'
PRECISION_LABEL = 'Precision | Positive Predictive Value | TP/(TP+FP)'

SUBPLOT_SIZE = 8

COLOR_ARRAY = [
    'tan', 'indigo', 'cyan', 'pink', 'purple', 'blue', 'chartreuse', 'deepskyblue', 'green', 'salmon', 'aqua', 'magenta', 'aquamarine', 'red',
    'coral', 'tomato', 'grey', 'black', 'maroon', 'hotpink', 'steelblue', 'orange', 'papayawhip', 'wheat', 'chocolate', 'darkkhaki', 'gold',
    'orange', 'crimson', 'slategray', 'violet', 'cadetblue', 'midnightblue', 'darkorchid', 'paleturquoise', 'plum', 'lime',
    'teal', 'peru', 'silver', 'darkgreen', 'rosybrown', 'firebrick', 'saddlebrown', 'dodgerblue', 'orangered',
]

import csv
import gzip
import h5py
import shutil
import zipfile
import pydicom
import numpy as np


# Keras imports
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ThresholdedReLU, Lambda, Reshape, LayerNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, add, concatenate
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten, LSTM, RepeatVector
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, UpSampling1D, UpSampling2D, UpSampling3D, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D, Layer
from tensorflow.keras.layers import SeparableConv1D, SeparableConv2D, DepthwiseConv2D


from ml4cvd.defines import StorageType
from ml4cvd.arguments import parse_args, TMAPS, _get_tmap
from ml4cvd.TensorMap import TensorMap, Interpretation
from ml4cvd.tensor_generators import test_train_valid_tensor_generators, big_batch_from_minibatch_generator
from ml4cvd.models import train_model_from_generators, make_multimodal_multitask_model, _inspect_model, train_model_from_generators, make_hidden_layer_model
from ml4cvd.recipes import test_multimodal_multitask, train_multimodal_multitask, saliency_maps, _predict_and_evaluate

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Constants
HD5_FOLDER = '/mnt/disks/ecg-rest-38k-tensors/2020-03-14/'

In [21]:
### classification task

sys.argv = ['train', 
            '--tensors', HD5_FOLDER, 
            '--input_tensors', 'ecg_rest',
            '--output_tensors', 'poor_data_quality',
            '--protected_tensors', 'sex', 'genetic_caucasian', 'age_0',
            '--training_steps', '96',
            '--validation_steps', '24',
            '--test_steps', '24',
            '--epochs', '6',
            '--batch_size', '24',
            '--id', 'ecg_rest_bias'
           ]

args = parse_args()

2020-07-14 13:27:44,353 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_rest_bias/log_2020-07-14_13-27_0.log.
2020-07-14 13:27:44,355 - arguments:418 - INFO - Command Line was: 
./scripts/tf.sh train --tensors /mnt/disks/ecg-rest-38k-tensors/2020-03-14/ --input_tensors ecg_rest --output_tensors poor_data_quality --protected_tensors sex genetic_caucasian age_0 --training_steps 96 --validation_steps 24 --test_steps 24 --epochs 6 --batch_size 24 --id ecg_rest_bias

2020-07-14 13:27:44,356 - arguments:419 - INFO - Total TensorMaps: 6534 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, anneal_max=2.0, anneal_rate=0.0, anneal_shift=0.0, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=24, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, bottleneck_type=<BottleneckType.FlattenRestructu

In [13]:
### regression task

sys.argv = ['train', 
            '--tensors', HD5_FOLDER, 
            '--input_tensors', 'ecg_rest',
            '--output_tensors', 'ventricular-rate',
            '--protected_tensors', 'sex', 'genetic_caucasian', 'age_0',
            '--training_steps', '48',
            '--validation_steps', '24',
            '--test_steps', '24',
            '--epochs', '80',
            '--batch_size', '8',
            '--id', 'ecg_rest_bias',
            '--patience', '24'
           ]

args = parse_args()

2020-07-14 13:15:11,631 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_rest_bias/log_2020-07-14_13-15_0.log.
2020-07-14 13:15:11,633 - arguments:418 - INFO - Command Line was: 
./scripts/tf.sh train --tensors /mnt/disks/ecg-rest-38k-tensors/2020-03-14/ --input_tensors ecg_rest --output_tensors ventricular-rate --protected_tensors sex genetic_caucasian age_0 --training_steps 48 --validation_steps 24 --test_steps 24 --epochs 80 --batch_size 8 --id ecg_rest_bias --patience 24

2020-07-14 13:15:11,635 - arguments:419 - INFO - Total TensorMaps: 586 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, anneal_max=2.0, anneal_rate=0.0, anneal_shift=0.0, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=8, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, bottleneck_type=<BottleneckType.Flatt

In [22]:
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)

2020-07-14 13:27:55,257 - tensor_generators:651 - INFO - Found 25955 train, 7575 validation, and 3710 testing tensors at: /mnt/disks/ecg-rest-38k-tensors/2020-03-14/
2020-07-14 13:27:55,283 - tensor_generators:238 - INFO - Stopped 4 workers. 


In [23]:
model = make_multimodal_multitask_model(**args.__dict__)
train_model_from_generators(model, generate_train, generate_valid, args.training_steps, args.validation_steps, 
                            args.batch_size, args.epochs, args.patience, args.output_folder, args.id, 
                            args.inspect_model, args.inspect_show_labels)

2020-07-14 13:27:55,332 - models:808 - WARNING - Number of x dimensions for convolutional kernel sizes (1) do not match number of convolutional layers/blocks (4), matching values to fit 4 convolutional layers/blocks.
2020-07-14 13:27:55,342 - models:808 - WARNING - Number of y dimensions for convolutional kernel sizes (1) do not match number of convolutional layers/blocks (4), matching values to fit 4 convolutional layers/blocks.
2020-07-14 13:27:55,344 - models:808 - WARNING - Number of z dimensions for convolutional kernel sizes (1) do not match number of convolutional layers/blocks (4), matching values to fit 4 convolutional layers/blocks.
2020-07-14 13:27:55,345 - models:344 - INFO - Conv layer is <class 'tensorflow.python.keras.layers.convolutional.Conv1D'> and kernels [(3,)]
2020-07-14 13:27:55,347 - models:346 - INFO -  Filters is 32 and kernel is (3,) at step 0
2020-07-14 13:27:55,351 - models:358 - INFO - Residual Block Convolutional Layers (num_filters, kernel_size): [(32, (3

2020-07-14 13:27:56,027 - tensor_generators:151 - INFO - Started 3 train workers with cache size 0.875 GB.
2020-07-14 13:27:56,591 - tensor_generators:151 - INFO - Started 1 validation workers with cache size 0.875 GB.
Train for 96 steps, validate for 24 steps
Epoch 1/6
95/96 [============================>.] - ETA: 0s - loss: 0.1297 - categorical_accuracy: 0.8360 - no_poor_data_quality_precision: 0.9761 - Poor_data_quality_precision: 0.0986 - no_poor_data_quality_recall: 0.8522 - Poor_data_quality_recall: 0.2061
Epoch 00001: val_loss improved from inf to 0.10291, saving model to ./recipes_output/ecg_rest_bias/ecg_rest_bias.h5
96/96 [==============================] - 25s 256ms/step - loss: 0.1289 - categorical_accuracy: 0.8377 - no_poor_data_quality_precision: 0.9763 - Poor_data_quality_precision: 0.0976 - no_poor_data_quality_recall: 0.8537 - Poor_data_quality_recall: 0.2040 - val_loss: 0.1029 - val_categorical_accuracy: 0.9792 - val_no_poor_data_quality_precision: 0.9808 - val_Poor_da

<Figure size 1728x1152 with 0 Axes>

In [27]:
def evaluate_predictions(
    tm: TensorMap, y_predictions: np.ndarray, y_truth: np.ndarray, protected: Dict[TensorMap, np.ndarray], title: str, folder: str, test_paths: List[str] = None,
    max_melt: int = 150000, rocs: List[Tuple[np.ndarray, np.ndarray, Dict[str, int]]] = [],
    scatters: List[Tuple[np.ndarray, np.ndarray, str, List[str]]] = [],
) -> Dict[str, float]:
    """ Evaluate predictions for a given TensorMap with truth data and plot the appropriate metrics.
    Accumulates data in the rocs and scatters lists to facilitate subplotting.

    :param tm: The TensorMap predictions to evaluate
    :param y_predictions: The predictions
    :param y_truth: The truth
    :param title: A title for the plots
    :param folder: The folder to save the plots at
    :param test_paths: The tensor paths that were predicted
    :param max_melt: For multi-dimensional prediction the maximum number of prediction to allow in the flattened array
    :param protected: TensorMaps and tensors sensitive to bias
    :param rocs: (output) List of Tuples which are inputs for ROC curve plotting to allow subplotting downstream
    :param scatters: (output) List of Tuples which are inputs for scatter plots to allow subplotting downstream
    :return: Dictionary of performance metrics with string keys for labels and float values
    """
    performance_metrics = {}
    if tm.is_categorical() and tm.axes() == 1:
        logging.info(f"For tm:{tm.name} with channel map:{tm.channel_map} examples:{y_predictions.shape[0]}")
        logging.info(f"\nSum Truth:{np.sum(y_truth, axis=0)} \nSum pred :{np.sum(y_predictions, axis=0)}")
        performance_metrics.update(subplot_roc_per_class(y_predictions, y_truth, tm.channel_map, protected, 
                                                      title, folder))
        rocs.append((y_predictions, y_truth, tm.channel_map))
    elif tm.is_continuous() and tm.axes() ==1:
        performance_metrics.update(subplot_pearson_per_class(y_predictions, y_truth, tm.channel_map, protected, 
                                                      title, folder))

    return performance_metrics


def get_fpr_tpr_roc_pred(y_pred, test_truth, labels):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for k in labels:
        cur_idx = labels[k]
        aser = roc_curve(test_truth[:, cur_idx], y_pred[:, cur_idx])
        fpr[labels[k]], tpr[labels[k]], _ = aser
        roc_auc[labels[k]] = auc(fpr[labels[k]], tpr[labels[k]])

    return fpr, tpr, roc_auc
def _hash_string_to_color(string):
    """Hash a string to color (using hashlib and not the built-in hash for consistency between runs)"""
    return COLOR_ARRAY[int(hashlib.sha1(string.encode('utf-8')).hexdigest(), 16) % len(COLOR_ARRAY)]


def _text_on_plot(axes, x, y, text, alpha=0.8, background='white'):
    t = axes.text(x, y, text)
    t.set_bbox({'facecolor': background, 'alpha': alpha, 'edgecolor': background})


    

def new_predict_and_evaluate(model, test_data, test_labels, tensor_maps_in, tensor_maps_out, 
                             tensor_maps_protected, batch_size, hidden_layer, plot_path, 
                             test_paths, embed_visualization, alpha):
    layer_names = [layer.name for layer in model.layers]
    performance_metrics = {}
    scatters = []
    rocs = []
    
    protected_data = {tm: test_labels[tm.output_name()] for tm in tensor_maps_protected}
    print(f'tm prot {len(protected_data)}')
    
    y_predictions = model.predict(test_data, batch_size=batch_size)
    for y, tm in zip(y_predictions, tensor_maps_out):
        if tm.output_name() not in layer_names:
            continue
        if not isinstance(y_predictions, list):  # When models have a single output model.predict returns a ndarray otherwise it returns a list
            y = y_predictions
        y_truth = np.array(test_labels[tm.output_name()])
        performance_metrics.update(evaluate_predictions(tm, y, y_truth, protected_data, tm.name, plot_path, 
                                                        test_paths, rocs=rocs, scatters=scatters))
        if tm.is_language():
            sample_from_language_model(tensor_maps_in, tm, model, test_data, max_samples=16)

    if len(rocs) > 1:
        subplot_rocs(rocs, plot_path)
    if len(scatters) > 1:
        subplot_scatters(scatters, plot_path)

    test_labels_1d = {tm: np.array(test_labels[tm.output_name()]) for tm in tensor_maps_out if tm.output_name() in test_labels}
    if embed_visualization == "tsne":
        _tsne_wrapper(model, hidden_layer, alpha, plot_path, test_paths, test_labels_1d, test_data=test_data, tensor_maps_in=tensor_maps_in, batch_size=batch_size)

    return performance_metrics

In [28]:
def subplot_roc_per_class(prediction, truth, labels, protected, title, prefix='./figures/'):
    lw = 2
    col = 0
    row = 1
    labels_to_areas = {}
    true_sums = np.sum(truth, axis=0)
    total_plots = len(protected) + 1
    cols = max(2, int(math.ceil(math.sqrt(total_plots))))
    rows = max(2, int(math.ceil(total_plots / cols)))
    fig, axes = plt.subplots(rows, cols, figsize=(cols*SUBPLOT_SIZE, rows*SUBPLOT_SIZE))
    fpr, tpr, roc_auc = get_fpr_tpr_roc_pred(prediction, truth, labels)
    
    for p in protected:
        print(f'\n name {p.name} truth shape {truth.shape} IN ROCCCC {p.name} and {p.shape} and {protected[p].shape}')
        
        axes[row, col].plot([0, 1], [0, 1], 'k:', lw=0.5)
        axes[row, col].set_title(f'Protected {p.name}')
        for key in labels:    
            if p.is_categorical():
                idx2key = {v: k for k, v in p.channel_map.items()}
                protected_indexes = protected[p][:, 0] == 1
                print(f'\n\n protected_indexes shape {protected_indexes.shape}')

                pfpr, ptpr, proc_auc = get_fpr_tpr_roc_pred(prediction[protected_indexes], 
                                                            truth[protected_indexes], labels)
                label_text = f'{key} roc={proc_auc[labels[key]]:.3f} n={np.sum(protected_indexes):.0f}'

                color = _hash_string_to_color(p.name+key)
                axes[row, col].plot(pfpr[labels[key]], ptpr[labels[key]], color=color, lw=lw, label=label_text)
            elif p.is_continuous():
                threshold = np.median(protected[p])
                protected_indexes = (protected[p] > threshold)[:, 0]
                pfpr, ptpr, proc_auc = get_fpr_tpr_roc_pred(prediction[protected_indexes], 
                                                                truth[protected_indexes], labels)
                label_text = f'{key} roc={proc_auc[labels[key]]:.3f} Highest  n={np.sum(protected_indexes):.0f}'
                color = _hash_string_to_color(p.name+key)
                axes[row, col].plot(pfpr[labels[key]], ptpr[labels[key]], color=color, lw=lw, label=label_text)
                print(f'\n\n median {threshold} protected_indexes shape {protected[p].shape}')                
                axes[row, col].set_xlim([0.0, 1.0])
        axes[row, col].set_ylim([-0.02, 1.03])
        axes[row, col].set_ylabel(RECALL_LABEL)
        axes[row, col].set_xlabel(FALLOUT_LABEL)
        axes[row, col].legend(loc='lower right')
        row += 1
        if row == rows:
            row = 0
            col += 1
            if col >= cols:
                break
                    
    for key in labels:
        labels_to_areas[key] = roc_auc[labels[key]]
        if 'no_' in key and len(labels) == 2:
            continue
        color = _hash_string_to_color(key)
        label_text = f'{key} area: {roc_auc[labels[key]]:.3f} n={true_sums[labels[key]]:.0f}'
        axes[0, 0].plot(fpr[labels[key]], tpr[labels[key]], color=color, lw=lw, label=label_text)
        logging.info(f'ROC Label {label_text} Truth shape {truth.shape}, true sums {true_sums}')

    axes[0, 0].set_title(f'ROC {title} n={truth.shape[0]:.0f}\n')
    axes[0, 0].legend(loc='lower right')
    figure_path = os.path.join(prefix, 'per_class_roc_' + title + IMAGE_EXT)
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path, bbox_inches='tight')
    plt.clf()
    logging.info(f"Saved ROC curve at: {figure_path} with {len(protected)} protected TensorMaps.")
    return labels_to_areas

In [29]:
def subplot_pearson_per_class(prediction, truth, labels, protected, title, prefix='./figures/'):
    
    ### labels are tm.channel_map. Looks like this: {'no_poor_data_quality': 0, 'Poor data quality': 1}
    
    lw = 2
    col = 0
    row = 1
    alpha=0.5
    labels_to_areas = {}
    true_sums = np.sum(truth, axis=0)
    total_plots = len(protected) + 1 
    cols = max(2, int(math.ceil(math.sqrt(total_plots))))
    rows = max(2, int(math.ceil(total_plots / cols)))
    fig, axes = plt.subplots(rows, cols, figsize=(cols*SUBPLOT_SIZE, rows*SUBPLOT_SIZE))

    for p in protected: 

        axes[row, col].plot([0, 1], [0, 1], 'k:', lw=0.5)
        axes[row, col].set_title(f'Protected {p.name}')
        for key in labels:    
            if p.is_categorical():
                idx2key = {v: k for k, v in p.channel_map.items()}
                protected_indexes = protected[p][:, 0] == 1 
                print(f'\n\n protected_indexes shape {protected_indexes.shape}')
                color = _hash_string_to_color(p.name+key)
                axes[row, col].plot([np.min(truth), np.max(truth)], [np.min(truth), np.max(truth)], linewidth=2)
                axes[row, col].plot([np.min(prediction), np.max(prediction)], [np.min(prediction), np.max(prediction)], linewidth=4)
                pearson = np.corrcoef(prediction[protected_indexes].flatten(), truth[protected_indexes].flatten())[1, 0]
                big_r_squared = coefficient_of_determination(truth[protected_indexes], prediction[protected_indexes])
                axes[row, col].scatter(prediction[protected_indexes], truth[protected_indexes], color=color, lw=lw,label=f'Pearson:{pearson:0.3f} r^2:{pearson*pearson:0.3f} R^2:{big_r_squared:0.3f} Highest n={np.sum(protected_indexes):.0f}', marker='.', alpha=alpha)


            elif p.is_continuous(): #### top/bottom quantile
                threshold = np.median(protected[p])
                protected_indexes = (protected[p] > threshold)[:, 0]
                color = _hash_string_to_color(p.name+key)
                axes[row, col].plot([np.min(truth), np.max(truth)], [np.min(truth), np.max(truth)], linewidth=2)
                axes[row, col].plot([np.min(prediction), np.max(prediction)], [np.min(prediction), np.max(prediction)], linewidth=4)
                pearson = np.corrcoef(prediction[protected_indexes].flatten(), truth[protected_indexes].flatten())[1, 0]
                big_r_squared = coefficient_of_determination(truth[protected_indexes], prediction[protected_indexes])
                axes[row, col].scatter(prediction[protected_indexes], truth[protected_indexes], color=color, lw=lw,label=f'Pearson:{pearson:0.3f} r^2:{pearson*pearson:0.3f} R^2:{big_r_squared:0.3f} Highest n={np.sum(protected_indexes):.0f}', marker='.', alpha=alpha)



                print(f'\n\n median {threshold} protected_indexes shape {protected[p].shape}')


        axes[row, col].set_ylabel('Predictions')
        axes[row, col].set_xlabel('Actual')
        axes[row, col].legend(loc='lower right')
        row += 1
        if row == rows:
            row = 0
            col += 1
            if col >= cols:
                break

            
            
    axes[0,0].plot([np.min(truth), np.max(truth)], [np.min(truth), np.max(truth)], linewidth=2)
    axes[0,0].plot([np.min(prediction), np.max(prediction)], [np.min(prediction), np.max(prediction)], linewidth=4)
    pearson = np.corrcoef(prediction.flatten(), truth.flatten())[1, 0]
    big_r_squared = coefficient_of_determination(truth, prediction)  
    label_text = f'Pearson:{pearson:0.3f} r^2:{pearson*pearson:0.3f} R^2:{big_r_squared:0.3f} n={truth.shape[0]:.0f}'
    axes[0, 0].scatter(prediction, truth, color=color, lw=lw,label=label_text, marker='.', alpha=alpha)
    axes[0, 0].legend(loc='lower right')
    axes[0, 0].set_title(f'Pearson {title}')

    figure_path = os.path.join(prefix, 'per_class_pearson_' + title + IMAGE_EXT)
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path, bbox_inches='tight')
    plt.clf()
    logging.info(f"Saved Pearson correlations at: {figure_path} with {len(protected)} protected TensorMaps.")
    return labels_to_areas

In [30]:
out_path = os.path.join(args.output_folder, args.id + '/')
test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
new_predict_and_evaluate(model, test_data, test_labels, args.tensor_maps_in, args.tensor_maps_out, 
                      args.tensor_maps_protected, args.batch_size, args.hidden_layer, out_path, 
                      test_paths, args.embed_visualization, args.alpha)

2020-07-14 13:35:45,919 - tensor_generators:151 - INFO - Started 4 test workers with cache size 0.000 GB.
2020-07-14 13:35:52,071 - tensor_generators:504 - INFO - Made a big batch of tensors with key:input_strip_continuous and shape:(576, 5000, 12).
2020-07-14 13:35:52,073 - tensor_generators:504 - INFO - Made a big batch of tensors with key:output_poor_data_quality_categorical and shape:(576, 2).
2020-07-14 13:35:52,075 - tensor_generators:504 - INFO - Made a big batch of tensors with key:output_Sex_Male_0_0_categorical and shape:(576, 2).
2020-07-14 13:35:52,076 - tensor_generators:504 - INFO - Made a big batch of tensors with key:output_Genetic-ethnic-grouping_Caucasian_0_0_categorical and shape:(576, 2).
2020-07-14 13:35:52,078 - tensor_generators:504 - INFO - Made a big batch of tensors with key:output_21003_Age-when-attended-assessment-centre_0_0_continuous and shape:(576, 1).
tm prot 3
2020-07-14 13:35:53,250 - <ipython-input-27-bc1ddedc3fe6>:23 - INFO - For tm:poor_data_quality

{'no_poor_data_quality': 0.853713606654783,
 'Poor data quality': 0.8537136066547831}

<Figure size 1152x1152 with 0 Axes>